In [23]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
import re
from nltk.corpus import stopwords
import nltk.data
from gensim.models import word2vec, Word2Vec
import logging
from sklearn.ensemble import RandomForestClassifier
from __future__ import print_function

nltk.download('stopwords')

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jg/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
punkt_cięcia = 30000
wymiary = 1000

In [25]:
def tekst_na_listę_słów(tekst, remove_stopwords=False):
    # Usuń tagi HTML
    tekst = BeautifulSoup(tekst, "lxml").get_text()
    
    # Usuń znaki spoza alfabetu; wszystko na wersaliki
    tekst = re.sub("[^a-zA-Z]", " ", tekst).lower()

    # Do listy
    słowa = tekst.lower().split()
    
    # Usuń "stopwords"
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        słowa = [w for w in słowa if not w in stops]
    
    return(słowa)

In [26]:
def słowa_na_wektory(lista_słów, model):
    lista_wektorów = []
    
    zbiór_index2word = set(model.wv.index2word)
    
    for słowo in lista_słów:
        if słowo in zbiór_index2word:
            lista_wektorów += [model[słowo]]
            
    return lista_wektorów

In [33]:
def przetwórz_na_wektory(teksty, model):
    licznik = 1
    
    teksty_wektorowo = []
    
    for tekst in teksty:
        if licznik%1000 == 0:
            print("Recenzja %d z %d"%(licznik, len(teksty)))
            
        teksty_wektorowo += [słowa_na_wektory(tekst, model)]
        licznik += 1
        
    return teksty_wektorowo

In [28]:
dane = pd.read_csv('/home/jg/Uczenie maszynowe/Zbiory/filmy_dane.csv')
dane

,Recenzja,Sentyment
0,Another great musical from Hollywoods Golden A...,1
1,...but I would be lying. A relative was a crew...,0
2,One of the worst movies I've seen shoddy camer...,0
3,I'm going to review the 2 films as a whole bec...,1
4,Wealthy widower Anthony Steffen (as Alan Cunni...,0
5,"Everyone in the cast, from Sugiyama to Aoki an...",1
6,This is a FUNNY film. It has all the usual Dis...,1
7,I had heard interesting critics on this movie....,1
8,An interesting look at Japan prior to opening ...,1
9,"The movie ""Holly"" is the story of a young girl...",1


In [29]:
model = Word2Vec.load('/home/jg/Uczenie maszynowe/imdb2.bin')

In [30]:
dane_treningowe = dane[:punkt_cięcia]
dane_testowe = dane[punkt_cięcia:]

In [34]:
recenzje_treningowe = []
for recenzja in dane_treningowe['Recenzja']:
    recenzje_treningowe.append(tekst_na_listę_słów(recenzja, remove_stopwords=True))
    
wektory_treningowe = przetwórz_na_wektory(recenzje_treningowe, model)

/home/jg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Recenzja 1000 z 30000
Recenzja 2000 z 30000
Recenzja 3000 z 30000
Recenzja 4000 z 30000
Recenzja 5000 z 30000
Recenzja 6000 z 30000
Recenzja 7000 z 30000
Recenzja 8000 z 30000
Recenzja 9000 z 30000
Recenzja 10000 z 30000
Recenzja 11000 z 30000
Recenzja 12000 z 30000
Recenzja 13000 z 30000
Recenzja 14000 z 30000
Recenzja 15000 z 30000
Recenzja 16000 z 30000
Recenzja 17000 z 30000
Recenzja 18000 z 30000
Recenzja 19000 z 30000
Recenzja 20000 z 30000
Recenzja 21000 z 30000
Recenzja 22000 z 30000
Recenzja 23000 z 30000
Recenzja 24000 z 30000
Recenzja 25000 z 30000
Recenzja 26000 z 30000
Recenzja 27000 z 30000
Recenzja 28000 z 30000
Recenzja 29000 z 30000
Recenzja 30000 z 30000


In [35]:
recenzje_testowe = []
for recenzja in dane_testowe['Recenzja']:
    recenzje_testowe.append(tekst_na_listę_słów(recenzja, remove_stopwords=True))

In [37]:
wektory_testowe = przetwórz_na_wektory(recenzje_testowe, model)

/home/jg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Recenzja 1000 z 20000
Recenzja 2000 z 20000
Recenzja 3000 z 20000
Recenzja 4000 z 20000
Recenzja 5000 z 20000
Recenzja 6000 z 20000
Recenzja 7000 z 20000
Recenzja 8000 z 20000
Recenzja 9000 z 20000
Recenzja 10000 z 20000
Recenzja 11000 z 20000
Recenzja 12000 z 20000
Recenzja 13000 z 20000
Recenzja 14000 z 20000
Recenzja 15000 z 20000
Recenzja 16000 z 20000
Recenzja 17000 z 20000
Recenzja 18000 z 20000
Recenzja 19000 z 20000
Recenzja 20000 z 20000


[array([-2.40051690e-02,  1.26336357e-02,  5.60885621e-03, -2.07623132e-02,
        -3.51011716e-02,  4.06557173e-02, -2.40663998e-02, -5.48051372e-02,
        -2.69692987e-02,  3.43286134e-02,  5.37947146e-03,  5.07686101e-02,
        -5.78645943e-03,  1.97661594e-02, -9.10329632e-04, -5.63942827e-02,
        -1.18776802e-02,  3.22823003e-02,  2.05072295e-02, -7.61638908e-03,
         8.43743503e-04, -2.14795619e-02,  2.65791193e-02, -1.61590707e-02,
        -1.92863513e-02,  1.08970720e-02,  7.61899026e-03,  3.21120620e-02,
         9.30132158e-03,  1.68467481e-02,  2.51826812e-02,  2.57886257e-02,
         1.17772548e-02, -1.94796883e-02,  3.43637317e-02, -2.31230482e-02,
        -5.75088412e-02,  4.01567109e-02,  9.61741753e-05, -5.67713380e-02,
         7.57798599e-03, -4.53865416e-02,  3.02141849e-02, -4.08455171e-02,
         7.05348328e-02, -4.89401557e-02, -1.15995362e-01, -2.39365175e-02,
         2.96384227e-02,  5.14977351e-02, -1.26649374e-02,  6.11755028e-02,
         1.0